In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool, Tool

# 3.5는 결과가 틀림
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0.1)

# 하나의 input만 받기 때문에 문자열을 split 하고, 이걸 description에 설명해줌.
def plus(inputs):
    a, b = inputs.split(",")
    return float(a) + float(b)

"""
agent에 input을 줄 때 runtime은 LLM으로 부터 get_action을 호출해서 LLM이 선택한 Tool을 돌림
AgentFinish가 될떄까지 loop를 돌리기 떄문에 Cost가 많이 나올 수 있음.
가끔 LLM이 Json이 아닌 Text로 답할 때가 있음. 그러땐 Agent가 에러가 남
"""
# llm, tool, agent를 전달해야 함.
agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, # 하나의 Input을 가진 tool을 invoke 하는 agent
    handle_parsing_errors=True, # LLM의 output 때문에 Parsing Error가 발생하면 이를 수정하려고 함.
    tools=[
        # 함수와 그에 대한 설명을 전달
        Tool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. Use this tool by sending a pair of number separated by a comma.\nExample:1,2",
        ),
    ],
)


# LLM은 산수를 직접 할 수 없음
prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12" # $5,273.38

# LangChain이 tool을 포함한 prompt를 만들어서 LLM에 전달함.
agent.invoke(prompt) # $4923.38